In [0]:
#Imports
from pyspark.sql.functions import *

#configuration
spark.conf.set(
  "fs.azure.account.key.ecommercestorage5400.dfs.core.windows.net",
    "<storage_account_access_key>"
)

#dest path
gold_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/gold"
#src path
silver_path = "abfss://ecommerce@ecommercestorage5400.dfs.core.windows.net/silver"

#create from silver
df_silver = (
    spark.readStream
    .format('delta')
    .load(silver_path)
)

#aggregation: Total sales and total items sold per state per minute
df_gold = (
    df_silver
    .withWatermark('timestamp','1 minute')
    .groupBy(
        window('timestamp','1 minute'),
        'state')
    .agg(
        sum('total_amount').alias('total_sales'),
        sum('quantity').alias('total_items')
    )
    .select(
        col('window.start').alias('start'),
        col('window.end').alias('end'),
        'state',
        'total_sales',
        'total_items'
    )
)

(
    df_gold.writeStream
    .format('delta')
    .outputMode('append')
    .option('checkpointLocation', f'{gold_path}/_checkpoint')
    .start(gold_path)
)

In [0]:
df_gold =spark.readStream.format('delta').load(gold_path)
display(df_gold.limit(10))

start,end,state,total_sales,total_items
2025-11-24T17:05:00Z,2025-11-24T17:06:00Z,NY,10597.599999999999,7
2025-11-24T12:58:00Z,2025-11-24T12:59:00Z,CA,5009.299999999999,6
2025-11-24T12:30:00Z,2025-11-24T12:31:00Z,NY,11872.880000000001,15
2025-11-24T16:46:00Z,2025-11-24T16:47:00Z,NY,21981.670000000002,17
2025-11-25T04:01:00Z,2025-11-25T04:02:00Z,IL,34463.49,22
2025-11-25T03:30:00Z,2025-11-25T03:31:00Z,CA,30032.769999999997,22
2025-11-25T03:16:00Z,2025-11-25T03:17:00Z,AZ,7434.72,9
2025-11-25T02:53:00Z,2025-11-25T02:54:00Z,CA,6933.03,7
2025-11-24T11:51:00Z,2025-11-24T11:52:00Z,AZ,8049.2699999999995,15
2025-11-25T03:22:00Z,2025-11-25T03:23:00Z,CA,22001.92,21
